In [4]:
import pennylane as qml
from pennylane import numpy as np
import pennylane.qaoa.cost # Import the module containing the conversion function

In [5]:
# --- 1. Define QUBO Parameters ---

# Configuration
M = 5         # Number of qubits (layers)
B = 3         # Budget (target number of layers to select)
LAMBDA = 100  # Penalty coefficient (lambda)
AVG_WEIGHT = 0.25 # Average model priority (w_bar)

# Relative Layer Importance (Cl) - Estimated based on parameter count contribution
# [L1, L2, L3, L4, L5] = [linear, transformer, conv, dense1, dense2]
C_l = np.array([0.05, 0.50, 0.15, 0.20, 0.10])

# --- 2. Construct the QUBO Matrix (Q) ---
Q = np.zeros((M, M))

# Diagonal Terms (Q_l,l)
diagonal_base = LAMBDA * (1 - 2 * B)
Q[np.diag_indices(M)] = diagonal_base - C_l * AVG_WEIGHT

# Off-Diagonal Terms (Q_l,k)
off_diagonal_value = 2 * LAMBDA
for i in range(M):
    for j in range(i + 1, M):
        Q[i, j] = off_diagonal_value
        Q[j, i] = off_diagonal_value

print("--- QUBO Matrix (Q) ---")
print(Q)

# --- CORRECTED LINE: Convert QUBO Matrix to Ising Hamiltonian (H) ---
# Use the precise path: pennylane.qaoa.cost.qubo_to_ising
H, _ = pennylane.qaoa.cost.qubo_to_ising(Q) 

print("\n--- Ising Hamiltonian (H) constructed from QUBO ---")
print(H)

# --- 3. QAOA Implementation with PennyLane ---

# Set up the quantum device
dev = qml.device("default.qubit", wires=M)

# Number of QAOA layers (p)
p = 2 

# Define the QAOA ansatz (circuit)
@qml.qnode(dev, interface='autograd')
def qaoa_circuit(gamma, beta):
    # Apply Hadamard to all qubits for the initial state superposition
    for i in range(M):
        qml.Hadamard(wires=i)
    
    # QAOA layers
    for layer in range(p):
        # 1. Cost Hamiltonian evolution (U_C)
        qml.ApproxTimeEvolution(H, gamma[layer], 1)
        
        # 2. Mixer Hamiltonian evolution (U_M) - Standard X-Mixer
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i)
    
    # Return the expectation value of the Cost Hamiltonian (H)
    return qml.expval(H)

# Initialize the optimizer and parameters
optimizer = qml.AdamOptimizer(stepsize=0.1)
# Initialize gamma and beta parameters randomly
params = 0.5 * np.random.rand(2, p)
num_steps = 100 # Number of optimization steps

# Optimization loop
print("\n--- Running QAOA Optimization (p=2, steps=100) ---")
for step in range(num_steps):
    params, cost = optimizer.step_and_cost(qaoa_circuit, params)
    
    if (step + 1) % 20 == 0:
        print(f"Step {step+1}: Cost = {cost:.4f}")

print(f"\nOptimization finished. Final Cost: {cost:.4f}")

# --- 4. Solution Extraction and Layer Selection ---

# Get the most likely classical state by sampling the circuit
@qml.qnode(dev, interface='autograd')
def sampling_circuit(gamma, beta):
    # Re-run the optimized QAOA circuit
    for i in range(M):
        qml.Hadamard(wires=i)
    for layer in range(p):
        qml.ApproxTimeEvolution(H, gamma[layer], 1)
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i)
    
    # Return the probability distribution over all 2^M states
    return qml.probs(wires=range(M))

# Get the probability distribution
probs = sampling_circuit(params[0], params[1])

# Find the most probable state (the layer selection)
best_state_index = np.argmax(probs)
# Convert index (integer) to binary string, then to list of integers [x0, x1, x2, x3, x4]
selected_layers_binary = [int(bit) for bit in format(best_state_index, f'0{M}b')]
selected_layers_binary.reverse() # Reverse to match x0, x1, ... indexing

layer_names = ["L1: linear_projection", "L2: transformer_encoder", "L3: conv_layer", "L4: dense1", "L5: dense2"]

# --- 5. Final Result ---

print("\n--- Layer Selection Results ---")
print(f"Optimal Binary Solution (x): {selected_layers_binary}")
print(f"Number of layers selected: {sum(selected_layers_binary)}")
print(f"Target Budget (B): {B}")

print("\n**Selected Layers for Aggregation:**")
for i, is_selected in enumerate(selected_layers_binary):
    if is_selected:
        print(f"✅ {layer_names[i]} (Priority Weight: {C_l[i]:.2f})")
    else:
        print(f"❌ {layer_names[i]} (Priority Weight: {C_l[i]:.2f})")

--- QUBO Matrix (Q) ---
[[-500.0125  200.      200.      200.      200.    ]
 [ 200.     -500.125   200.      200.      200.    ]
 [ 200.      200.     -500.0375  200.      200.    ]
 [ 200.      200.      200.     -500.05    200.    ]
 [ 200.      200.      200.      200.     -500.025 ]]


AttributeError: module 'pennylane.qaoa.cost' has no attribute 'qubo_to_ising'

In [6]:
import pennylane as qml
from pennylane import numpy as np

# --- 1. Define QUBO Parameters ---

# Configuration
M = 5         # Number of qubits (layers)
B = 3         # Budget (target number of layers to select)
LAMBDA = 100  # Penalty coefficient (lambda)
AVG_WEIGHT = 0.25 # Average model priority (w_bar)

# Relative Layer Importance (Cl) - Estimated based on parameter count contribution
# [L1, L2, L3, L4, L5] = [linear, transformer, conv, dense1, dense2]
C_l = np.array([0.05, 0.50, 0.15, 0.20, 0.10])

# --- 2. Construct the QUBO Matrix (Q) ---
Q = np.zeros((M, M))

# Diagonal Terms (Q_l,l)
diagonal_base = LAMBDA * (1 - 2 * B) # 100 * (-5) = -500
Q[np.diag_indices(M)] = diagonal_base - C_l * AVG_WEIGHT

# Off-Diagonal Terms (Q_l,k)
off_diagonal_value = 2 * LAMBDA # 200
for i in range(M):
    for j in range(i + 1, M):
        Q[i, j] = off_diagonal_value
        Q[j, i] = off_diagonal_value

print("--- QUBO Matrix (Q) ---")
print(Q)

# --- 3. Manual Conversion to Ising Hamiltonian (H) ---
# Conversion rules: J_ij = Q_ij / 4; h_i = -Q_ii/2 - sum(Q_ij)/4 for j!=i

coeffs = []
obs = []

# Linear terms (h_i Z_i)
for i in range(M):
    # Sum of off-diagonal terms: Q_ij / 4 for j != i
    sum_off_diag = np.sum(Q[i, :]) - Q[i, i] # Sum all except Q_i,i
    h_i = -Q[i, i] / 2 - sum_off_diag / 4
    coeffs.append(h_i)
    obs.append(qml.PauliZ(i))

# Quadratic terms (J_ij Z_i Z_j)
for i in range(M):
    for j in range(i + 1, M):
        J_ij = Q[i, j] / 4
        coeffs.append(J_ij)
        obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

H = qml.Hamiltonian(coeffs, obs)

print("\n--- Ising Hamiltonian (H) constructed manually ---")
print(H)

# --- 4. QAOA Implementation with PennyLane ---

dev = qml.device("default.qubit", wires=M)
p = 2 # Number of QAOA layers

@qml.qnode(dev, interface='autograd')
def qaoa_circuit(gamma, beta):
    for i in range(M):
        qml.Hadamard(wires=i)
    
    for layer in range(p):
        qml.ApproxTimeEvolution(H, gamma[layer], 1) # Cost Hamiltonian
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i) # Mixer Hamiltonian
    
    return qml.expval(H)

# Optimization loop
optimizer = qml.AdamOptimizer(stepsize=0.1)
params = 0.5 * np.random.rand(2, p)
num_steps = 100 

print("\n--- Running QAOA Optimization (p=2, steps=100) ---")
for step in range(num_steps):
    params, cost = optimizer.step_and_cost(qaoa_circuit, params)
    
    if (step + 1) % 20 == 0:
        print(f"Step {step+1}: Cost = {cost:.4f}")

print(f"\nOptimization finished. Final Cost: {cost:.4f}")

# --- 5. Solution Extraction and Layer Selection ---

@qml.qnode(dev, interface='autograd')
def sampling_circuit(gamma, beta):
    # Re-run the optimized QAOA circuit
    for i in range(M):
        qml.Hadamard(wires=i)
    for layer in range(p):
        qml.ApproxTimeEvolution(H, gamma[layer], 1)
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i)
    
    return qml.probs(wires=range(M))

probs = sampling_circuit(params[0], params[1])

# Find the most probable state (the layer selection)
best_state_index = np.argmax(probs)
# Convert index (integer) to binary string, then to list of integers [x0, x1, x2, x3, x4]
selected_layers_binary = [int(bit) for bit in format(best_state_index, f'0{M}b')]
selected_layers_binary.reverse() # Reverse to match x0, x1, ... indexing

layer_names = ["L1: linear_projection", "L2: transformer_encoder", "L3: conv_layer", "L4: dense1", "L5: dense2"]
C_l_print = [0.05, 0.50, 0.15, 0.20, 0.10] # Use standard list for printing

# --- 6. Final Result ---

print("\n--- Layer Selection Results ---")
print(f"Optimal Binary Solution (x): {selected_layers_binary}")
print(f"Number of layers selected: {sum(selected_layers_binary)} (Target Budget: {B})")

print("\n**Selected Layers for Aggregation:**")
for i, is_selected in enumerate(selected_layers_binary):
    if is_selected:
        print(f"✅ {layer_names[i]} (Priority Weight: {C_l_print[i]:.2f})")
    else:
        print(f"❌ {layer_names[i]} (Priority Weight: {C_l_print[i]:.2f})")

--- QUBO Matrix (Q) ---
[[-500.0125  200.      200.      200.      200.    ]
 [ 200.     -500.125   200.      200.      200.    ]
 [ 200.      200.     -500.0375  200.      200.    ]
 [ 200.      200.      200.     -500.05    200.    ]
 [ 200.      200.      200.      200.     -500.025 ]]

--- Ising Hamiltonian (H) constructed manually ---
50.006249999999994 * Z(0) + 50.0625 * Z(1) + 50.01875000000001 * Z(2) + 50.025000000000006 * Z(3) + 50.01249999999999 * Z(4) + 50.0 * (Z(0) @ Z(1)) + 50.0 * (Z(0) @ Z(2)) + 50.0 * (Z(0) @ Z(3)) + 50.0 * (Z(0) @ Z(4)) + 50.0 * (Z(1) @ Z(2)) + 50.0 * (Z(1) @ Z(3)) + 50.0 * (Z(1) @ Z(4)) + 50.0 * (Z(2) @ Z(3)) + 50.0 * (Z(2) @ Z(4)) + 50.0 * (Z(3) @ Z(4))

--- Running QAOA Optimization (p=2, steps=100) ---


TypeError: qaoa_circuit() missing 1 required positional argument: 'beta'

In [8]:
import pennylane as qml
from pennylane import numpy as np

# --- 1. Define QUBO Parameters ---

# Configuration
M = 5         # Number of qubits (layers)
B = 4         # Budget (target number of layers to select)
LAMBDA = 100  # Penalty coefficient (lambda)
AVG_WEIGHT = 0.25 # Average model priority (w_bar)
p = 2         # Number of QAOA layers

# Relative Layer Importance (Cl) - Estimated based on parameter count contribution
# [L1, L2, L3, L4, L5] = [linear, transformer, conv, dense1, dense2]
C_l = np.array([0.05, 0.50, 0.15, 0.20, 0.10])
layer_names = ["L1: linear_projection", "L2: transformer_encoder", "L3: conv_layer", "L4: dense1", "L5: dense2"]


# --- 2. Construct the QUBO Matrix (Q) ---
Q = np.zeros((M, M))

# Diagonal Terms (Q_l,l)
diagonal_base = LAMBDA * (1 - 2 * B) # -500
Q[np.diag_indices(M)] = diagonal_base - C_l * AVG_WEIGHT

# Off-Diagonal Terms (Q_l,k)
off_diagonal_value = 2 * LAMBDA # 200
for i in range(M):
    for j in range(i + 1, M):
        Q[i, j] = off_diagonal_value
        Q[j, i] = off_diagonal_value

print("--- QUBO Matrix (Q) ---")
print(Q)

# --- 3. Manual Conversion to Ising Hamiltonian (H) ---
# H = sum h_i Z_i + sum J_ij Z_i Z_j
# Conversion rules: J_ij = Q_ij / 4; h_i = -Q_ii/2 - sum(Q_ij)/4 for j!=i

coeffs = []
obs = []

# Linear terms (h_i Z_i)
for i in range(M):
    # Sum of off-diagonal terms: Q_ij / 4 for j != i
    sum_off_diag = np.sum(Q[i, :]) - Q[i, i]
    h_i = -Q[i, i] / 2 - sum_off_diag / 4
    coeffs.append(h_i)
    obs.append(qml.PauliZ(i))

# Quadratic terms (J_ij Z_i Z_j)
for i in range(M):
    for j in range(i + 1, M):
        J_ij = Q[i, j] / 4
        coeffs.append(J_ij)
        obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

H = qml.Hamiltonian(coeffs, obs)

print("\n--- Ising Hamiltonian (H) constructed manually ---")
print(H)

# --- 4. QAOA Circuit Definition ---

dev = qml.device("default.qubit", wires=M)

@qml.qnode(dev, interface='autograd')
def qaoa_circuit(gamma, beta):
    # Initial state (Superposition)
    for i in range(M):
        qml.Hadamard(wires=i)
    
    # QAOA layers
    for layer in range(p):
        qml.ApproxTimeEvolution(H, gamma[layer], 1) # Cost Hamiltonian (U_C)
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i) # Mixer Hamiltonian (U_M)
    
    return qml.expval(H)

# --- 5. Optimization ---

# Initialize gamma and beta parameters as separate arrays (CORRECTION)
gamma_init = 0.5 * np.random.rand(p)
beta_init = 0.5 * np.random.rand(p)
qaoa_params = (gamma_init, beta_init) 

optimizer = qml.AdamOptimizer(stepsize=0.1)
num_steps = 100 

print("\n--- Running QAOA Optimization (p=2, steps=100) ---")
for step in range(num_steps):
    # Unpack the parameters using * to pass gamma and beta separately (CORRECTION)
    qaoa_params, cost = optimizer.step_and_cost(qaoa_circuit, *qaoa_params)
    
    if (step + 1) % 20 == 0:
        print(f"Step {step+1}: Cost = {cost:.4f}")

print(f"\nOptimization finished. Final Cost: {cost:.4f}")

# --- 6. Solution Extraction and Layer Selection ---

@qml.qnode(dev, interface='autograd')
def sampling_circuit(gamma, beta):
    # Re-run the optimized QAOA circuit
    for i in range(M):
        qml.Hadamard(wires=i)
    for layer in range(p):
        qml.ApproxTimeEvolution(H, gamma[layer], 1)
        for i in range(M):
            qml.RX(2 * beta[layer], wires=i)
    
    # Return the probability distribution over all 2^M states
    return qml.probs(wires=range(M))

probs = sampling_circuit(*qaoa_params) # Unpack parameters for sampling
best_state_index = np.argmax(probs)

# Convert index to binary string, then to list, and reverse for correct qubit order (x0, x1, ...)
selected_layers_binary = [int(bit) for bit in format(best_state_index, f'0{M}b')]
selected_layers_binary.reverse() 

# --- 7. Final Result ---

print("\n--- Layer Selection Results ---")
print(f"Optimal Binary Solution (x): {selected_layers_binary}")
print(f"Number of layers selected: {sum(selected_layers_binary)} (Target Budget: {B})")

print("\n**Selected Layers for Aggregation:**")
for i, is_selected in enumerate(selected_layers_binary):
    if is_selected:
        print(f"✅ {layer_names[i]} (Priority Weight: {C_l[i]:.2f})")
    else:
        print(f"❌ {layer_names[i]} (Priority Weight: {C_l[i]:.2f})")

--- QUBO Matrix (Q) ---
[[-700.0125  200.      200.      200.      200.    ]
 [ 200.     -700.125   200.      200.      200.    ]
 [ 200.      200.     -700.0375  200.      200.    ]
 [ 200.      200.      200.     -700.05    200.    ]
 [ 200.      200.      200.      200.     -700.025 ]]

--- Ising Hamiltonian (H) constructed manually ---
150.00625000000002 * Z(0) + 150.0625 * Z(1) + 150.01875 * Z(2) + 150.02499999999998 * Z(3) + 150.0125 * Z(4) + 50.0 * (Z(0) @ Z(1)) + 50.0 * (Z(0) @ Z(2)) + 50.0 * (Z(0) @ Z(3)) + 50.0 * (Z(0) @ Z(4)) + 50.0 * (Z(1) @ Z(2)) + 50.0 * (Z(1) @ Z(3)) + 50.0 * (Z(1) @ Z(4)) + 50.0 * (Z(2) @ Z(3)) + 50.0 * (Z(2) @ Z(4)) + 50.0 * (Z(3) @ Z(4))

--- Running QAOA Optimization (p=2, steps=100) ---
Step 20: Cost = 85.9985
Step 40: Cost = 93.0432
Step 60: Cost = 2.6336
Step 80: Cost = -229.6646
Step 100: Cost = -22.8642

Optimization finished. Final Cost: -22.8642

--- Layer Selection Results ---
Optimal Binary Solution (x): [1, 1, 1, 1, 1]
Number of layers sele